In [79]:
pip install playsound

In [80]:
pip install multiprocessing

In [5]:
import cv2
import numpy as np
import datetime, time
from collections import deque
import multiprocessing
from playsound import playsound

In [81]:
class Motion_Detection:
    def __init__(self, Video_Capture, deque_maxlen, frame_size):
        self.Size = frame_size
        
        _, first_frame = Video_Capture.read()
        first_frame = cv2.cvtColor(first_frame, cv2.COLOR_BGR2GRAY)
        first_frame = cv2.resize(first_frame, (self.Size, self.Size))
        first_frame_z = np.zeros_like(first_frame)
    
        self.d = deque(maxlen=deque_maxlen)
        for i in range(deque_maxlen):
            self.d.append(first_frame_z)
    
    
    def detect(self, frame):
        f_gray = cv2.cvtColor(frame.copy(), cv2.COLOR_BGR2GRAY)
        f_gray = cv2.resize(f_gray, (self.Size, self.Size))
        f_guss = cv2.GaussianBlur(f_gray, (7, 7), 0)
        
        self.d.append(f_guss)
    
        frame_diff1 = cv2.absdiff(self.d[-2], self.d[-1])
        frame_diff2 = cv2.absdiff(self.d[-3], self.d[-2])

        frame_thresh1 = cv2.threshold(frame_diff1, 30, 255, cv2.THRESH_BINARY)[1]
        frame_thresh2 = cv2.threshold(frame_diff2, 30, 255, cv2.THRESH_BINARY)[1]

        thresh_and = cv2.bitwise_and(frame_thresh1, frame_thresh2)
        return thresh_and
    
    def tracking(self, frame_detected, frame_org, Min_Area):
        self.cnts, _ = cv2.findContours(frame_detected.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        for c in self.cnts:
            if cv2.contourArea(c) < Min_Area:
                continue
            (x, y, w, h) = cv2.boundingRect(c)
            cv2.rectangle(frame_org, (x, y), (x + w, y + h), (0, 255, 0), 3)
            cv2.putText(frame_org, "Tracking", (15, 25), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)
       
        return frame_org
    
    def Warning(self, warning_TRUEorFALSE, frame_detected, frame_org, sleep):
        if warning_TRUEorFALSE == False:
            p = multiprocessing.Process(target=playsound, args=("warning.mp3",))
            countNoneZero = cv2.countNonZero(frame_detected)
            if countNoneZero != 0:
                time_ = datetime.datetime.now().strftime("-%H-%M")
                cv2.imwrite(f"tracking{time}.jpg", frame_org)
                p.start()
                t1 = time.sleep(sleep)
                p.terminate()
            

In [82]:
cap = cv2.VideoCapture(0)

motionOBJECT = Motion_Detection(cap, 3, 540)

while True:
    ret, frame = cap.read()
    detected = motionOBJECT.detect(frame)
    tracked = motionOBJECT.tracking(detected, frame, 0)

    motionOBJECT.Warning(True, detected, frame, 2)
    
    cv2.imshow("motion detection", tracked)
    if cv2.waitKey(1) == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()